In [1]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set LANGCHAIN_TRACING_V2 to "true"
os.environ["LANGCHAIN_TRACING_V2"] = "true"

# Set API keys from .env file
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["TVLY_API_KEY"] = os.getenv("TVLY_API_KEY")



In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1727805778, 'localtime': '2024-10-01 11:02'}, 'current': {'last_updated_epoch': 1727805600, 'last_updated': '2024-10-01 11:00', 'temp_c': 23.3, 'temp_f': 73.9, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 45, 'wind_dir': 'NE', 'pressure_mb': 1015.0, 'pressure_in': 29.98, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 60, 'cloud': 0, 'feelslike_c': 24.5, 'feelslike_f': 76.1, 'windchill_c': 23.7, 'windchill_f': 74.7, 'heatindex_c': 23.9, 'heatindex_f': 74.9, 'dewpoint_c': 9.0, 'dewpoint_f': 48.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 6.0, 'gust_mph': 11.7, 'gust_kph': 18.9}}"}, {'url': 'https://world-weather.info/forecast/usa/san_franc

In [8]:
pip install -qU langchain-openai 

Note: you may need to restart the kernel to use updated packages.


In [9]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

In [10]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [11]:
model_with_tools = model.bind_tools(tools)

In [12]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [13]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_aWHK2SveSIhNwf2rxhL6bj55', 'type': 'tool_call'}]


In [14]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [15]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', additional_kwargs={}, response_metadata={}, id='54281396-134e-40e4-b0a1-813bdb7349fa'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fac39566-4514-4421-ad16-567d168cfa7f-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [16]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='7b7355ee-0d5f-4f6f-8554-623864ac6ad3'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MttsQy2aYn8oBaDUppKBpXzy', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e4bd15a8-8ada-41c0-8afe-7d0a32199f67-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_MttsQy2aYn8oBaDUppKBpXzy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88

In [17]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_C1S0OIHLFt6snFmFhGJ1Z491', 'function': {'arguments': '{\n  "query": "current weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 88, 'total_tokens': 111, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b8b4532c-0510-4268-9521-f465bba6cff9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_C1S0OIHLFt6snFmFhGJ1Z491', 'type': 'tool_call'}], usage_metadata={'input_tokens': 88, 'output_tokens': 23, 'total_tokens': 111})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https:/

In [18]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

As| an| AI| model| developed| by| Open|AI|,| I| don|'t| have| real|-time| data| access|,| including| current| weather| updates|.| However|,| you| can| easily| check| the| current| weather| of| San| Francisco| by| typing| "|San| Francisco| weather|"| into| any| reliable| online| weather| forecasting| service| like| the| Weather| Channel|,| BBC| Weather|,| or| using| a| smart| home| device|.|

In [19]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [20]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [21]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1c495199-0573-4491-b552-6f1da838b3f8-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96})]}}
----


In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 108, 'total_tokens': 114, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b0af2d5a-ac84-4ff3-b4b4-57185b489697-0', usage_metadata={'input_tokens': 108, 'output_tokens': 6, 'total_tokens': 114})]}}
----


In [23]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="As an AI, I don't have access to personal data about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. So, I'm unable to tell you your name.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 86, 'total_tokens': 136, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-48a8124d-2697-4ba6-8bed-df03e58cbec5-0', usage_metadata={'input_tokens': 86, 'output_tokens': 50, 'total_tokens': 136})]}}
----
